# Add to the path

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

# Import Libraries

In [28]:
from Tensorized_Layers.TCL import TCL
from Utils.Num_parameter import count_parameters

import torch
import torch.nn as nn
import time

# Dummy Data

In [29]:
device = 'cuda'
batch_size = 5
dummy = torch.rand(batch_size, 14, 14, 16, 16, 3).to(device)
print(f'Current shape is : {dummy.shape}')

Current shape is : torch.Size([5, 14, 14, 16, 16, 3])


# TCL
- input size is (5, 14, 14, 16, 16, 3)
- ranks are (16,16,3) # excluding the ignored modes
- ignored modes are 0: batch , 1:patch index 1, 2: patch index 2
- device is set to cuda
- output shape is ignore modes + ranks

In [30]:
tcl = TCL(input_size=(5,14,14,16,16,3),
            rank=(16,16,3),
            ignore_modes=(0,1,2),
            bias = True,
            device='cuda'
            ).to('cuda')
st = time.time()
output = tcl(dummy)
elapsed = time.time() - st
print(f'output shape of tcl: {output.shape}')

print(f'This tcl has {count_parameters(tcl)} parameters')
print(f'This tcl took : {elapsed}')

output shape of tcl: torch.Size([5, 14, 14, 16, 16, 3])
This tcl has 1289 parameters
This tcl took : 0.000270843505859375


In [31]:
for key in tcl.state_dict():
    print(key, tcl.state_dict()[key].device)

b cuda:0
factors.0 cuda:0
factors.1 cuda:0
factors.2 cuda:0


In [32]:
temp_loss = nn.CrossEntropyLoss()
temp_y = torch.randint(0, 14, (output.shape[0], output.shape[2], output.shape[3], output.shape[4], output.shape[5])).to(device)
l = temp_loss(output, temp_y)
l.backward()
# If no error, Then is fine

# Compare to tltorch TRL from Tensorly

In [33]:
device = 'cuda'
batch_size = 5
dummy = torch.rand(batch_size, 16, 16, 3).to(device)
print(f'Current shape is : {dummy.shape}')

Current shape is : torch.Size([5, 16, 16, 3])


## Forward Pass

In [34]:
tcl1 = TCL(input_size=dummy.shape,
            rank=(16,16,3),
            ignore_modes=(0,),
            bias=False,
            device = device
            ).to(device)
st = time.time()
output1 = tcl1(dummy)
elapsed = time.time() - st
print(f'output shape of tcl (our method): {output1.shape}')

print(f'This tcl (our method) has {count_parameters(tcl1)} parameters')
print(f'This tcl (our method) took : {elapsed}')

output shape of tcl (our method): torch.Size([5, 16, 16, 3])
This tcl (our method) has 521 parameters
This tcl (our method) took : 0.00022101402282714844


In [36]:
import tltorch # doesnt work with bias

tcl2 = tltorch.TCL(input_shape = (16,16,3), 
                   bias = False,
                   rank = (16,16,3,)).to(device)
st = time.time()
output2 = tcl2(dummy)
elapsed = time.time() - st
print(f'output shape of trl (tltorch method): {output2.shape}')

print(f'This trl (tltorch method) has {count_parameters(tcl2)} parameters')
print(f'This trl (tltorch method) took : {elapsed}')

output shape of trl (tltorch method): torch.Size([5, 16, 16, 3])
This trl (tltorch method) has 521 parameters
This trl (tltorch method) took : 0.0013196468353271484


## Backward Pass

In [37]:
temp_loss = nn.CrossEntropyLoss()
temp_y = torch.randint(0, 16, (output1.shape[0], output1.shape[2], output1.shape[3])).to(device)
st = time.time()
l = temp_loss(output1, temp_y)
l.backward()
elapsed = time.time() - st
print(f'This backward trl (our method) took : {elapsed}')

This backward trl (our method) took : 0.0006608963012695312


In [38]:
temp_loss = nn.CrossEntropyLoss()
temp_y = torch.randint(0, 16, (output2.shape[0], output2.shape[2], output2.shape[3])).to(device)
st = time.time()
l = temp_loss(output2, temp_y)
l.backward()
elapsed = time.time() - st
print(f'This backward trl (tltorch method) took : {elapsed}')

This backward trl (tltorch method) took : 0.0005712509155273438
